![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use Watsonx to analyze car rental customer satisfaction and offer recommendation.

**Note:** Please note that for the watsonx challenge, please run these notebooks in IBM Cloud and not on on your laptop/desktop.

This notebook contains the steps and code to demonstrate support of text sentiment analysis in Watsonx. It introduces commands for data retrieval, model testing and scoring.

Some familiarity with Python is helpful. This notebook uses Python 3.10.

<a id="setup"></a>
## Set up the environment

### Install and import the dependecies

In [1]:
!pip install datasets | tail -n 1
!pip install scikit-learn | tail -n 1
!pip install ibm-watson-machine-learning==1.0.312 | tail -n 1

**Note:** Please restart the notebook kernel to pick up proper version of packages installed above.

In [150]:
import os, getpass
from pandas import read_csv, concat

### Watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud user API key. Instructions have been provided to generate IBM Cloud API key. For details, see
[documentation](https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui).

In [145]:
from ibm_cloud_sdk_core import IAMTokenManager
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator, BearerTokenAuthenticator
import os, getpass

access_token = IAMTokenManager(
#     apikey = getpass.getpass("Please enter your api key (hit enter): "),
    apikey = "EQKPblVNI4xAQsorly666ngHKKd4oGxECdByjRuSD16b",
    url = "https://iam.cloud.ibm.com/identity/token"
).get_token()

### Defining the project id
The API requires project id that provides the context for the call. We will obtain the id from the project in which this notebook runs. When you run notebook on IBM Cloud, project in which it runs is saved as environment variable PROJECT_ID.

**Hint**: You can find the `project_id` as follows. Open the prompt lab in watsonx.ai. At the very top of the UI, there will be `Projects / <project name> /`. Click on the `<project name>` link. Then get the `project_id` from Project's Manage tab (Project -> Manage -> General -> Details).


In [3]:
try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

<a id="data"></a>
## Train/test data loading

Load train and test datasets. At first, training dataset (`train_data`) should be used to work with the models to prepare and tune prompt. Then, test dataset (`test_data`) should be used to calculate the metrics score for selected model, defined prompts and parameters.

In [164]:
filename_test = 'https://watsonx-gsi-challenge.s3.jp-tok.cloud-object-storage.appdomain.cloud/track1/test.csv'
filename_train = 'https://watsonx-gsi-challenge.s3.jp-tok.cloud-object-storage.appdomain.cloud/track1/train.csv'

test_data = read_csv(filename_test)
train_data = read_csv(filename_train)
all_data = concat([train_data, test_data])
print(all_data.shape[0])
use_data = test_data

150


In [5]:
train_data.head()

,ID,Gender,Status,Children,Age,Customer_Status,Car_Owner,Customer_Service,Satisfaction,Business_Area,Action
0,2944,Female,M,2,41.92,Active,No,Customer service was friendly and helpful.,1,Service: Knowledge,NaN
1,1119,Female,M,2,33.60,Active,Yes,Customer service was good at MSP airport and t...,1,Service: Knowledge,NaN
2,0,Male,M,0,51.00,Inactive,Yes,I do not understand why I have to pay additio...,0,Product: Pricing and Billing,Premium features
3,1085,Female,S,2,42.00,Inactive,No,Based on the customer service personnel I enco...,0,Service: Attitude,On-demand pickup location
4,0,Female,M,2,44.10,Active,No,Provide more convenient car pickup from the ai...,0,Service: Orders/Contracts,On-demand pickup location


In [6]:
test_data.head()

,ID,Gender,Status,Children,Age,Customer_Status,Car_Owner,Customer_Service,Satisfaction,Business_Area,Action
0,2771,Female,M,2,49.99,Inactive,No,"last time I rented a car was at Manchester, NH...",0,Product: Functioning,On-demand pickup location
1,1133,Male,S,1,56.05,Inactive,No,Please lower the prices.,0,Product: Pricing and Billing,Free Upgrade
2,900,Female,M,1,64.64,Active,No,Excellent response dealing with child seat.,1,Service: Accessibility,NaN
3,3795,Male,M,0,46.51,Inactive,No,"all went quite smoothly... it was Enterprise, ...",1,Service: Accessibility,NaN
4,3541,Male,S,1,17.01,Inactive,Yes,"Slow, long lineup",0,Product: Functioning,On-demand pickup location


<a id="models"></a>
## Foundation Models on Watsonx



Below code invokes Watson Machine Learning API to invoke Watsonx.ai LLMs


In [89]:
import requests

class Prompt:
    def __init__(self, access_token, project_id):
        self.access_token = access_token
        self.project_id = project_id

    def generate(self, input, model_id, parameters):
        wml_url = "https://us-south.ml.cloud.ibm.com/ml/v1-beta/generation/text?version=2023-05-28"
        Headers = {
            "Authorization": "Bearer " + self.access_token,
            "Content-Type": "application/json",
            "Accept": "application/json"
        }
        data = {
            "model_id": model_id,
            "input": input,
            "parameters": parameters,
            "project_id": self.project_id
        }
        response = requests.post(wml_url, json=data, headers=Headers)
        if response.status_code == 200:
            return response.json()["results"][0]["generated_text"].strip()
        else:
            return response.text

<a id="predict"></a>
## Evaluate the model, prompt and parameters

### **1. Customer satisfaction**

Define instructions for the model to recognize if customer was satisfied or unsatisfied.

**Note:** Please **start with using [watsonx.ai Prompt Lab](https://dataplatform.cloud.ibm.com/wx/home?context=wx)** to find better prompts that provides you the best result on a small subset training records (under `train_data` variable). Make sure to not run an inference of all of `train_data`, as it'll take a long time to get the results. To get a sample from `train_data`, you can use e.g.`train_data.head(n=10)` to get first 10 records, or `train_data.sample(n=10)` to get random 10 records. Only once you have identified the best performing prompt, update this notebook to use the prompt and compute the metrics on the test data.

**Action:** Please edit the below cell and add your own prompt here. In the below prompt, we have the instruction (first sentence) and one example included in the prompt.  If you want to change the prompt or add your own examples or more examples, please change the below prompt accordingly.

In [168]:
satisfaction_instruction = """
Classify the sentiment of the comment as either "no" or "yes"

comment: I have had a few recent rentals that have taken a very very long time, with no offer of apology.  In the most recent case, the agent subsequently offered me a car type on an upgrade coupon and then told me it was no longer available because it had just be\n
satisfaction: no

comment: Person very friendly but only person working counter
satisfaction: yes

comment: it was fine, I had no problem at all.
satisfaction: yes

comment: My experience was positive .The thing I didnt like was returning the car with full tank.  It was time consuming, but I didn't want to pay fill-up charge to the rental company.
satisfaction: yes
"""

### Defining the model parameters
We need to provide a set of model parameters that will influence the result. We will use IBM's Granite model.

In [28]:
parameters = {
    "decoding_method": "greedy",
    "max_new_tokens": 10,
    "min_new_tokens": 0,
    "repetition_penalty": 1
}

model_id = "ibm/granite-13b-instruct-v1"

Analyze the customer satisfaction for inputs from the test set.

**Note:** Execution of this cell could take several minutes.

In [ ]:
results = []
prompt = Prompt(access_token, project_id)
comments = list(use_data.Customer_Service)
satisfaction = ['no' if x == '0' else 'yes' for x in list(use_data.Satisfaction.astype(str))]

# print(comments)
for idx, input_text, in enumerate(comments):
    prompt_template = satisfaction_instruction + "\ncomment: " + input_text + "\n\nsatisfaction:"
    print(prompt_template)
#     prompt_form = " ".join([satisfaction_instruction, input_text])
    generated = prompt.generate(prompt_template, model_id, parameters)
#     results.append(prompt.generate(" ".join([satisfaction_instruction, input_text, "\n"]), model_id, parameters))
#     print("\n\nPROMPT:", prompt_form,"\nRESULT:", generated, "\nExpected:", satisfaction[idx])
    results.append(generated)
    if generated != satisfaction[idx]:
        print("\nRESULT:", generated, "\nExpected:", satisfaction[idx])


Classify the sentiment of the comment as either "no" or "yes"

comment: I have had a few recent rentals that have taken a very very long time, with no offer of apology.  In the most recent case, the agent subsequently offered me a car type on an upgrade coupon and then told me it was no longer available because it had just be

satisfaction: no

comment: Person very friendly but only person working counter
satisfaction: yes

comment: it was fine, I had no problem at all.
satisfaction: yes

comment: My experience was positive .The thing I didnt like was returning the car with full tank.  It was time consuming, but I didn't want to pay fill-up charge to the rental company.
satisfaction: yes

comment: last time I rented a car was at Manchester, NH airport and they do not have office there anymore

satisfaction:

Classify the sentiment of the comment as either "no" or "yes"

comment: I have had a few recent rentals that have taken a very very long time, with no offer of apology.  In the mo


Classify the sentiment of the comment as either "no" or "yes"

comment: I have had a few recent rentals that have taken a very very long time, with no offer of apology.  In the most recent case, the agent subsequently offered me a car type on an upgrade coupon and then told me it was no longer available because it had just be

satisfaction: no

comment: Person very friendly but only person working counter
satisfaction: yes

comment: it was fine, I had no problem at all.
satisfaction: yes

comment: My experience was positive .The thing I didnt like was returning the car with full tank.  It was time consuming, but I didn't want to pay fill-up charge to the rental company.
satisfaction: yes

comment: It was okay, we got the car quickly, which is the most important thing

satisfaction:

Classify the sentiment of the comment as either "no" or "yes"

comment: I have had a few recent rentals that have taken a very very long time, with no offer of apology.  In the most recent case, the agent 


RESULT: no 
Expected: yes

Classify the sentiment of the comment as either "no" or "yes"

comment: I have had a few recent rentals that have taken a very very long time, with no offer of apology.  In the most recent case, the agent subsequently offered me a car type on an upgrade coupon and then told me it was no longer available because it had just be

satisfaction: no

comment: Person very friendly but only person working counter
satisfaction: yes

comment: it was fine, I had no problem at all.
satisfaction: yes

comment: My experience was positive .The thing I didnt like was returning the car with full tank.  It was time consuming, but I didn't want to pay fill-up charge to the rental company.
satisfaction: yes

comment: fast service

satisfaction:

Classify the sentiment of the comment as either "no" or "yes"

comment: I have had a few recent rentals that have taken a very very long time, with no offer of apology.  In the most recent case, the agent subsequently offered me a car t

### Calculate the F1 micro score

In [166]:
from sklearn.metrics import f1_score, confusion_matrix, classification_report
import numpy as np

# print(satisfaction,"\n", results)
print(confusion_matrix(satisfaction, results))
print(classification_report(satisfaction,results, digits=3, zero_division=1, labels=['yes','no']))
print('f1_micro_score', f1_score(satisfaction, results, average='micro', zero_division=1))



[[18  0]
 [ 3 29]]
              precision    recall  f1-score   support

         yes      1.000     0.906     0.951        32
          no      0.857     1.000     0.923        18

    accuracy                          0.940        50
   macro avg      0.929     0.953     0.937        50
weighted avg      0.949     0.940     0.941        50

f1_micro_score 0.94


### **2. Offer Recommendation**

Define instructions for the model to recommend best offer to an unsatisfied customer.

**Note:** Please **start with using [watsonx.ai Prompt Lab](https://dataplatform.cloud.ibm.com/wx/home?context=wx)** to find better prompts that provides you the best result on a small subset training records (under `train_data` variable). Make sure to not run an inference of all of `train_data`, as it'll take a long time to get the results. To get a sample from `train_data`, you can use e.g.`train_data.head(n=10)` to get first 10 records, or `train_data.sample(n=10)` to get random 10 records. Only once you have identified the best performing prompt, update this notebook to use the prompt and compute the metrics on the test data.

**Action:** Please edit the below cell and add your own prompt here. In the below prompt, we have the instruction (first sentence) and one example included in the prompt.  If you want to change the prompt or add your own examples or more examples, please change the below prompt accordingly.

In [143]:
on_demand = test_data.loc[test_data['Action']=='On-demand pickup location']
sample_on_demand = on_demand.head(n=1)
# print(list(sample_on_demand.Customer_Service))
# print(test_data['Action'])

filtered_df = test_data[test_data['Action'].notna()].drop_duplicates(subset=['Action'])
comment_index = filtered_df.columns.get_loc('Customer_Service')
action_list = list(filtered_df.Action)
comment_list = list(filtered_df.Customer_Service)
# get Customer_Service and Action as a list

offer_recommendation_instruction = "Generate next best offer to unsatisfied customer. Choose offer recommendation from the following: 'On-demand pickup location', 'Free Upgrade', 'Voucher', 'Premium features'."
for idx, value in enumerate(action_list):
    offer_recommendation_instruction += f"\n\ncomment: {comment_list[idx]}\noffer recommended: {value}"

print(offer_recommendation_instruction)

# offer_recommendation_instruction = """
#    Generate next best offer to unsatisfied customer. Choose offer recommendation from the following list: 'On-demand pickup location', 'Free Upgrade', 'Voucher', 'Premium features'.\n

#    comment: The company was overwhelmed by the number of customers verse the number of available agents and they were not articulating their situation to the customers well enough. I think we waited for almost 3 hours just to get a rental car. It was ridiculous.\n
#    offer recommended: 'On-demand pickup location'\n\n
# """

Generate next best offer to unsatisfied customer. Choose offer recommendation from the following: 'On-demand pickup location', 'Free Upgrade', 'Voucher', 'Premium features'.

comment: last time I rented a car was at Manchester, NH airport and they do not have office there anymore
offer recommended: On-demand pickup location

comment: Please lower the prices.
offer recommended: Free Upgrade

comment: Customer is important for the enjoyment of the car.  If it's a bad experience we won't return to that company if we can avoid it - they should remember abotut this
offer recommended: Voucher

comment: Agents usually try to get you to take insurance and pay for full tank of gas.  Some persist even after a "no thanks."  Even after noting preferences on my frequent user file that says I don't want them.
offer recommended: Premium features


### Defining the model parameters
We need to provide a set of model parameters that will influence the result. We will use IBM's Granite model.

In [93]:
parameters = {
    "decoding_method": "greedy",
    "max_new_tokens": 30,
    "min_new_tokens": 1,
    "repetition_penalty": 1
}

model_id = "ibm/granite-13b-instruct-v1"

Filter test data for unsatisfied customer

In [157]:
unsatisfied_test_data = all_data.loc[all_data['Satisfaction'] == 0]
unsatisfied_test_data.head(n=30)

,ID,Gender,Status,Children,Age,Customer_Status,Car_Owner,Customer_Service,Satisfaction,Business_Area,Action
2,0,Male,M,0,51.00,Inactive,Yes,I do not understand why I have to pay additio...,0,Product: Pricing and Billing,Premium features
3,1085,Female,S,2,42.00,Inactive,No,Based on the customer service personnel I enco...,0,Service: Attitude,On-demand pickup location
4,0,Female,M,2,44.10,Active,No,Provide more convenient car pickup from the ai...,0,Service: Orders/Contracts,On-demand pickup location
6,0,Female,M,2,44.03,Active,No,VERY slow service!,0,Service: Accessibility,Free Upgrade
8,0,Male,S,2,20.40,Inactive,No,They could really try work harder.,0,Service: Attitude,Free Upgrade
9,0,Female,S,2,40.50,Inactive,No,I would like the personnel to pretend they car...,0,Service: Attitude,On-demand pickup location
16,2501,Female,M,1,63.93,Inactive,No,"The customer service agent was helpful, but I ...",0,Product: Functioning,On-demand pickup location
18,1190,Female,M,1,52.15,Inactive,No,I had to wait in line for a long time to get a...,0,Product: Functioning,Voucher
19,2312,Male,S,2,53.28,Inactive,No,It was absolutely ATROCIOUS! My wife and I wer...,0,Product: Functioning,Voucher
21,1360,Female,S,0,48.53,Active,Yes,they should not try so hard to up sell,0,Service: Attitude,Free Upgrade


Analyze the recommended actions for inputs from the test set.

**Note:** Execution of this cell could take several minutes.

In [158]:
results = []
prompt = Prompt(access_token, project_id)
comments = list(unsatisfied_test_data.Customer_Service)
offer_recommended = list(unsatisfied_test_data.Action.astype(str))

for idx,input_text in enumerate(comments):
#     prompt_template = " ".join([offer_recommendation_instruction, input_text])
    prompt_template = offer_recommendation_instruction + "\n\ncomment: " + input_text + "\noffer recommended:"
    gen = prompt.generate(prompt_template, model_id, parameters)
    print("\n\n", prompt_template,"\nResult:", gen, "\nExpected:", offer_recommended[idx])
    results.append(gen)
    
# print(comments)
# print(offer_recommended)



 Generate next best offer to unsatisfied customer. Choose offer recommendation from the following: 'On-demand pickup location', 'Free Upgrade', 'Voucher', 'Premium features'.

comment: last time I rented a car was at Manchester, NH airport and they do not have office there anymore
offer recommended: On-demand pickup location

comment: Please lower the prices.
offer recommended: Free Upgrade

comment: Customer is important for the enjoyment of the car.  If it's a bad experience we won't return to that company if we can avoid it - they should remember abotut this
offer recommended: Voucher

comment: Agents usually try to get you to take insurance and pay for full tank of gas.  Some persist even after a "no thanks."  Even after noting preferences on my frequent user file that says I don't want them.
offer recommended: Premium features

comment: I do not  understand why I have to pay additional fee if vehicle is returned without a full tank.
offer recommended: 
Result: Voucher 
Expected:



 Generate next best offer to unsatisfied customer. Choose offer recommendation from the following: 'On-demand pickup location', 'Free Upgrade', 'Voucher', 'Premium features'.

comment: last time I rented a car was at Manchester, NH airport and they do not have office there anymore
offer recommended: On-demand pickup location

comment: Please lower the prices.
offer recommended: Free Upgrade

comment: Customer is important for the enjoyment of the car.  If it's a bad experience we won't return to that company if we can avoid it - they should remember abotut this
offer recommended: Voucher

comment: Agents usually try to get you to take insurance and pay for full tank of gas.  Some persist even after a "no thanks."  Even after noting preferences on my frequent user file that says I don't want them.
offer recommended: Premium features

comment: It was absolutely ATROCIOUS! My wife and I were in a foreign country  when we realized that our car had an expired license plate and expired pro



 Generate next best offer to unsatisfied customer. Choose offer recommendation from the following: 'On-demand pickup location', 'Free Upgrade', 'Voucher', 'Premium features'.

comment: last time I rented a car was at Manchester, NH airport and they do not have office there anymore
offer recommended: On-demand pickup location

comment: Please lower the prices.
offer recommended: Free Upgrade

comment: Customer is important for the enjoyment of the car.  If it's a bad experience we won't return to that company if we can avoid it - they should remember abotut this
offer recommended: Voucher

comment: Agents usually try to get you to take insurance and pay for full tank of gas.  Some persist even after a "no thanks."  Even after noting preferences on my frequent user file that says I don't want them.
offer recommended: Premium features

comment: It would be nice if they included maps to the airport drop off with the car, last time we got lost returning the car and almost missed the fligh



 Generate next best offer to unsatisfied customer. Choose offer recommendation from the following: 'On-demand pickup location', 'Free Upgrade', 'Voucher', 'Premium features'.

comment: last time I rented a car was at Manchester, NH airport and they do not have office there anymore
offer recommended: On-demand pickup location

comment: Please lower the prices.
offer recommended: Free Upgrade

comment: Customer is important for the enjoyment of the car.  If it's a bad experience we won't return to that company if we can avoid it - they should remember abotut this
offer recommended: Voucher

comment: Agents usually try to get you to take insurance and pay for full tank of gas.  Some persist even after a "no thanks."  Even after noting preferences on my frequent user file that says I don't want them.
offer recommended: Premium features

comment: Having knowledgeable personnel is very important
offer recommended: 
Result: Free Upgrade 
Expected: Voucher


 Generate next best offer to unsa



 Generate next best offer to unsatisfied customer. Choose offer recommendation from the following: 'On-demand pickup location', 'Free Upgrade', 'Voucher', 'Premium features'.

comment: last time I rented a car was at Manchester, NH airport and they do not have office there anymore
offer recommended: On-demand pickup location

comment: Please lower the prices.
offer recommended: Free Upgrade

comment: Customer is important for the enjoyment of the car.  If it's a bad experience we won't return to that company if we can avoid it - they should remember abotut this
offer recommended: Voucher

comment: Agents usually try to get you to take insurance and pay for full tank of gas.  Some persist even after a "no thanks."  Even after noting preferences on my frequent user file that says I don't want them.
offer recommended: Premium features

comment: I thought that they were very short and not very friendly. I felt like they hated their job and could care less about the customer.
offer recomm



 Generate next best offer to unsatisfied customer. Choose offer recommendation from the following: 'On-demand pickup location', 'Free Upgrade', 'Voucher', 'Premium features'.

comment: last time I rented a car was at Manchester, NH airport and they do not have office there anymore
offer recommended: On-demand pickup location

comment: Please lower the prices.
offer recommended: Free Upgrade

comment: Customer is important for the enjoyment of the car.  If it's a bad experience we won't return to that company if we can avoid it - they should remember abotut this
offer recommended: Voucher

comment: Agents usually try to get you to take insurance and pay for full tank of gas.  Some persist even after a "no thanks."  Even after noting preferences on my frequent user file that says I don't want them.
offer recommended: Premium features

comment: last time I rented a car was at Manchester, NH airport and they do not have office there anymore
offer recommended: 
Result: On-demand pickup lo



 Generate next best offer to unsatisfied customer. Choose offer recommendation from the following: 'On-demand pickup location', 'Free Upgrade', 'Voucher', 'Premium features'.

comment: last time I rented a car was at Manchester, NH airport and they do not have office there anymore
offer recommended: On-demand pickup location

comment: Please lower the prices.
offer recommended: Free Upgrade

comment: Customer is important for the enjoyment of the car.  If it's a bad experience we won't return to that company if we can avoid it - they should remember abotut this
offer recommended: Voucher

comment: Agents usually try to get you to take insurance and pay for full tank of gas.  Some persist even after a "no thanks."  Even after noting preferences on my frequent user file that says I don't want them.
offer recommended: Premium features

comment: Customer service is horrible. I reserved through on-line which did not indicate/state I had to provide proof of insurance during pickup. The man



 Generate next best offer to unsatisfied customer. Choose offer recommendation from the following: 'On-demand pickup location', 'Free Upgrade', 'Voucher', 'Premium features'.

comment: last time I rented a car was at Manchester, NH airport and they do not have office there anymore
offer recommended: On-demand pickup location

comment: Please lower the prices.
offer recommended: Free Upgrade

comment: Customer is important for the enjoyment of the car.  If it's a bad experience we won't return to that company if we can avoid it - they should remember abotut this
offer recommended: Voucher

comment: Agents usually try to get you to take insurance and pay for full tank of gas.  Some persist even after a "no thanks."  Even after noting preferences on my frequent user file that says I don't want them.
offer recommended: Premium features

comment: the rep was friendly but it was so loud in there that I could not hear what she was saying. I HATE having to walk across a big lot with all of m

### Calculate the F1 micro score

In [160]:
from sklearn.metrics import f1_score

# print(offer_recommended)
# print(results)
print('f1_micro_score', f1_score(offer_recommended, results, average='micro'))

f1_micro_score 0.5084745762711864


---

Copyright © 2023 IBM. This notebook and its source code are released under the terms of the MIT License.